In [ ]:
# Lab 7.3

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
%matplotlib inline

# Lab Inferential statistics - T-test & P-value

# One tailed t-test - In a packing plant,  
a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

## Load data

In [11]:
machine = pd.read_csv('files_for_lab/machine.txt', encoding= 'utf-16', sep='\t')
machine.columns = ['new_machine','old_machine']
machine

# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

,new_machine,old_machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [13]:
type(machine.iloc[0,0])

numpy.float64

## Set hypothesis - pooled samples/independent samples  
* H0: new_machine < old_machine --> new_machine - old_machine < 0  
* H1: new_machine > old_machine --> new_machine - old_machine > 0  

For hypothesis testing purposes there is no need to compute the difference between samples as the samples are independent

In [14]:
machine['difference'] = machine.new_machine - machine.old_machine
machine

,new_machine,old_machine,difference
0,42.1,42.7,-0.6
1,41.0,43.6,-2.6
2,41.3,43.8,-2.5
3,41.8,43.3,-1.5
4,42.4,42.5,-0.1
5,42.8,43.5,-0.7
6,43.2,43.1,0.1
7,42.3,41.7,0.6
8,41.8,44.0,-2.2
9,42.7,44.1,-1.4


## Compute statistic

For hypothesis testing purposes there is no need to compute the difference between samples as the samples are independent

In [16]:
machine_diff_mean = machine.difference.mean()
machine_diff_mean

-1.0900000000000012

In [17]:
machine_diff_std = machine.difference.std(ddof=1)
machine_diff_std

1.125907041751967

In [62]:
new_machine_mean = machine.new_machine.mean()
new_machine_mean

42.14

In [64]:
new_machine_std = machine.new_machine.std()
new_machine_std

0.6834552736727638

In [65]:
old_machine_mean = machine.old_machine.mean()
old_machine_mean

43.230000000000004

In [66]:
old_machine_std = machine.old_machine.std()
old_machine_std

0.7498888806572157

In this case, **if we assume that the variances are equal** (1/2 < s1/s2 < 2) the t statistic is given by:

$$t=\frac{(\mu_{a}-\mu_{b})}{s_{p}\sqrt{\frac{1}{n_{a}}+\frac{1}{n_{b}}}}$$

with:

$$s_{p}=\sqrt{\frac{(n_{a}-1)s_{a}^{2}+(n_{b}-1)s_{b}^{2}}{(n_{a}+n_{b}-2)}}$$

$n_{a} = n_{b}$

$s_{p} = \sqrt{s_{a}^{2} s_{b}^{2}} = s_{a} s_{b} $

In [67]:
# s1/s2
old_machine_std/new_machine_std

1.0972025669323608

In [72]:
# sp
sp = np.sqrt((old_machine_std**2)*(new_machine_std**2))
sp

0.5125155101537399

In [73]:
machine_t = (new_machine_mean - old_machine_mean)/(sp*np.sqrt(2/len(machine)))
machine_t

-4.755590898592814

In [18]:
# wrong test: paired. Wrong t statistic

#machine_t = machine_diff_mean /(machine_diff_std/np.sqrt(len(machine)))
#machine_t

-3.0614273841115853

In [ ]:
# The critical statistic

In [70]:
machine_tc = st.t.ppf(1-0.05, df=len(machine)-1 )
machine_tc

1.8331129326536335

In [71]:
machine_t < machine_tc

True

Accept null hypothesis.  
The new machine is faster.  

# Matched Pairs Test  
In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

## Load the data

In [21]:
pokemon = pd.read_csv('files_for_lab/pokemon.csv')
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [22]:
pokemon[['Attack','Defense']].isna().sum()

Attack     0
Defense    0
dtype: int64

## Set the hypothesis

* H0: Attack = Defense --> Attack - Defense = 0
* H1: Attack <> Defense --> Attack - Defense <> 0

In [24]:
pokemon['difference'] = pokemon.Attack - pokemon.Defense
pokemon[['Attack','Defense','difference']]

,Attack,Defense,difference
0,49,49,0
1,62,63,-1
2,82,83,-1
3,100,123,-23
4,52,43,9
...,...,...,...
795,100,150,-50
796,160,110,50
797,110,60,50
798,160,60,100


## Compute statistic

In [25]:
pokemon_diff_mean = pokemon.difference.mean()
pokemon_diff_mean

5.15875

In [26]:
pokemon_diff_std = pokemon.difference.std(ddof=1)
pokemon_diff_std

33.7323418553516

In [27]:
pokemon_t = pokemon_diff_mean / (pokemon_diff_std/np.sqrt(len(pokemon)))
pokemon_t

4.325566393330483

In [28]:
pokemon_tc = st.t.ppf(1-(0.05/2), df=len(pokemon)-1)
pokemon_tc

1.9629374611056056

In [29]:
pokemon_t < pokemon_tc

False

Reject null hypothesis.  
Accept alternate hypothesis --> Attack <> Defense

## Another way  
ttest_rel function

In [32]:
st.ttest_rel(pokemon.Attack, pokemon.Defense)

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

In [33]:
pokemon_tc2, pokemon_pValue = st.ttest_rel(pokemon.Attack, pokemon.Defense)

In [35]:
(1-pokemon_pValue) < 1-(0.05/2)

False

Reject null hypothesis.  
Accept alternate hypothesis --> Attack <> Defense

# OPTIONAL PART | Inferential statistics - ANOVA

# Part 1
In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on:  
- Null hypothesis 
- Alternate hypothesis 
- Level of significance 
- Test statistic 
- P-value 
- F table



## Load data

In [37]:
etching = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
etching.columns = ['power','etching_rate']
etching

,power,etching_rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [38]:
etching.power[0]

'160 W'

## Set the hypothesis

$$H0: \mu_{1}=\mu_{2}=\mu_{3}=⋯=\mu_{n}$$
$$H1: \mu_{i}\neq\mu_{j}$$
for any i,j

If i did not misunderstand the changes in the power will not affect the means of the populatin according to the null hypothesis

Level of significance: 0.05

In case of the One way ANOVA test, the statistic to compute F is given by:

$$F = \frac{\hat{S}_{t}^{2}}{\hat{S}_{E}^{2}}$$

where:

$$\hat{S}_{t}^{2} = \frac{SST}{(K-1)}= \frac{\sum_{g}n_{g}(\bar{y}_{g}-\bar{y})^{2}}{(K-1)}$$

and:

$$\hat{S}_{E}^{2}=\frac{SSE}{(N-K)}=\frac{\sum_{g}\sum_{i}(y_{gi}-\bar{y}_{g})^{2}}{(N-K)}=\frac{\sum_{g}(n_{g}-1)s_{g}^{2}}{(N-K)}$$


The shape of the F distribution depends on two sets of degrees of freedom:  $d_{1}=K-1$ and $d_{2}=N-K$ 

In [40]:
# K
K = etching.power.nunique()
K

3

In [41]:
# d1
d1 = K -1
d1

2

In [43]:
# N
N = len(etching)
N

15

In [46]:
# d2 # Wrong it does not match with the scipy stats formula
# d2 = K*(N-1)
# d2

42

In [74]:
# d2 
d2 = N-K
d2

12

https://www.khanacademy.org/math/statistics-probability/analysis-of-variance-anova-library/analysis-of-variance-anova/v/anova-3-hypothesis-test-with-f-statistic

# Part 2
Conduct ANOVA with python.  

In [49]:
etching_group = etching.groupby('power')['etching_rate'].agg(rating_mean = 'mean', n = 'count').reset_index()
etching_group

,power,rating_mean,n
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


In [50]:
etching_mean = etching.etching_rate.mean()
etching_mean

6.782666666666667

## SSB  
Sum of squares between (or SST in class)

In [51]:
SSB = 0
for idx,power in enumerate(etching_group.power):
    SSB += etching_group.loc[idx,'n'] * (etching_group.loc[idx,'rating_mean']-etching_mean)**2
SSB

18.176653333333327

In [52]:
s2t = SSB/d1
s2t

9.088326666666664

## SSW  
Sum of squares within (or SSE in class)

In [53]:
SSW = 0
for idx,power in enumerate(etching_group.power):
    for rate in etching.etching_rate[etching.power == power]:
        SSW += (rate - etching_group.loc[idx,'rating_mean'])**2

SSW

2.9572399999999974

In [55]:
# Wrong: The value of d2 does not match the degrees of freedom for the scipy stats formula
# s2e = SSW/d2
# s2e

0.07041047619047613

In [75]:
s2e = SSW/d2
s2e

0.24643666666666644

## Compute F

In [57]:
# Wrong: s2e computed with theoretical degrees of freedom
#F = s2t/s2e
#F

129.07634145351753

In [76]:
F = s2t/s2e
F

36.878954701005014

In [59]:
# Critical F
Fc = st.f.ppf(0.95, dfn=d1, dfd=d2)
Fc

3.219942293176121

In [60]:
F < Fc

False

Reject null hypothesis.  
The mean of the population (etching_rate) is affected by the changes in the power.  
Accept alternate hypothesis

## Another Way  
f_oneway function

In [61]:
st.f_oneway(etching.etching_rate[etching.power == '160 W'],
            etching.etching_rate[etching.power == '180 W'],
            etching.etching_rate[etching.power == '200 W'])

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

It does not match the F computed manually (because of the change on the degrees of freedom d2).  
But gives the same result: reject null hypothesis

## Compute p-value

In [78]:
1-st.f.cdf(F, dfn = d1, dfd = d2)

7.5065842723986975e-06